# 比赛背景
实时环境感知是智能驾驶系统的重要组成部分。基于车载影像的目标检测和分割具有很高的价值，既能够辅助驾驶系统进行高效的决策，同时结合终端渲染与语音技术，也能够为用户带来更为智能精准的导航体验（比如AR导航）。 本次比赛数据由百度地图提供，要求在统一的计算资源下，快速识别道路关键可移动目标、交通标识和车道线。希望借助此次竞赛，充分发挥全球选手的聪明才智，共同推进智能驾驶技术的发展。

# 赛题说明
本次大赛提供10类共计20000张图像数据，其中1~7类采用矩形检测框标注方法，8~10类采用分割连通域标注方法，具体分类标准如下，要求用户最终对于图片的检测速度高于20fps

![](https://ai-studio-static-online.cdn.bcebos.com/701d88eadf8a4d83a22c99f1ba853737026d289d44fa4a59842814e09202afe5)



# 环境配置

1）预测框架：平台默认配置paddle（v2.1）、tensorflow（v2.1）和pytorch（v1.6）三类框架；

2）代码环境：平台仅配置python3.7执行环境；

3）环境库（可选）：选手如需使用超出平台的代码依赖库或框架，即要求定义好env目录，确保predict.py在平台能正常执行；

## 配置分析
Aistudio 用户平台默认配置了paddle,但是无法使用tensorflow和pytorch,预测框架则同时支持这三者.那么选手有两种选择:

1. 如果使用Aistudio平台进行训练的模型,那么在用户平台可以运行predict.py则在提交时一定可以在测试平台运行.

2. 如果不用Aistudio 平台进行训练,改用本地训练,使用tensorflow或者pytorch训练的模型,无法在用户平台运行predict.py. 如果本地可以运行predict.py,也可能在测试平台失败, 因为可能缺少相应的环境依赖,该错误在提交时是无法获取的.

本notebook是使用Aistudio的用户平台训练得到的模型进行测试的, 因此可以在用户平台可以获得与预测平台一致的结果

# 提交分析
AI Studio平台提供统一的预测机器资源（V100），并运行predict.py输出预测结果，具体执行命令如下：

In [ ]:
#predict.py data.txt result.json

## 提交重点
1) predict.py 输入：输入data.txt文件平台提供，文件分多行记录，每行对应一张图片路径，示例内容如下:

![](https://ai-studio-static-online.cdn.bcebos.com/374fe70cfa0a416890a1f31a5246e7cf18418bc39524458586f38bb607a414db)

2) predict.py 输出:预测结果需保存到result.json文件，格式要求如下：

![](https://ai-studio-static-online.cdn.bcebos.com/7adb85ea5614428ba5e1377d281c32856992a6151e0b4eea9b185dd07923ffd7)

3) FPS值

FPS = (预测图片总数)/(predict.py运行时间)

4) 最终提交

选手需上传训练模型、预测代码、训练代码及环境库（可选）的zip形式压缩包到AI Studio平台，其中要求model目录不超过200M（不压缩），整体压缩包不超过1G，目录结构约定如下：

![](https://ai-studio-static-online.cdn.bcebos.com/30d4d573c903423b9a90907ae068d88dbe1f41f358e346c185944bd8b05aa81a)

总结: 最终需要提交一个zip文件,解压后包括两个文件夹(env,model), 两个.py(predict.py, train.py).其中train.py不会被平台执行, predict.py会接受包含所有测试图片路径的data.txt文件,并将运行结果保存到result.json文件,文件格式如上图所示. model文件夹大小不超过200m, fps不低于20fps才能参与评分




# 实现思路

基于以上分析, 本文给出了使用PaddleDetection导出的模型进行预测的方案,这样的好处在于PaddleDetection本身具有丰富的模型库, 而且有现成的模型导出部署方案,非常方便进行迁移.
使用PaddleDetetion的export方法导出的模型结构如下:

![](https://ai-studio-static-online.cdn.bcebos.com/9ee334386f2a44feb384a8cf3c4b2c809a8903a607dd41fdbce4a583c7750d2d)

用tools/export.py 可以导出利用PaddleDetection 训练好的模型, 将该模型放入 model 文件夹之后,再吧所需要用到的依赖deploy文件夹放入env中 (本项目已经复制过了,使用者可以无需再复制),就可以通过修改PaddleDetection中的deploy/python/infer.py 来实现我们自己的predict.py了,其中的修改重点为:
1. 把图片的读取路径改成通过data.txt获取路径
2. 把模型的读取路径改成默认的model文件夹的路径(后文有具体修改方法)
4. 多设立一个result.json用来储存结果文件
5. 将之前的画检测图的函数改为生成结果文件,并将最终得到的结果存入result.json
下面是一个简单的利用ppyolo-tiny进行预测的实例

In [ ]:
# 解压训练集和测试集数据
!unzip -oq /home/aistudio/data/data94079/dataset.zip
!unzip -oq /home/aistudio/data/data94086/test_dataset.zip

warning [/home/aistudio/data/data94079/dataset.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
file #27590:  bad zipfile offset (local header sig):  44254
  (attempting to re-compensate)


In [ ]:
#安装预测依赖
!pip install paddledet==2.1.0 -i https://mirror.baidu.com/pypi/simple

In [ ]:
# 进入提交文件夹
%cd solutionSubmit/

/home/aistudio/solutionSubmit


In [ ]:
%cd model/

/home/aistudio/solutionSubmit/model


In [ ]:
# 把已经训练好的检测模型上传到model,文件为paddle detection 的 export导出的文件,并解压,保证解压之后的模型格式与本notebook相同,然乎需要去env.deploy.python.utils.py 修改模型默认路径,切记!
#!unzip ppyolo_tiny_650e_coco.zip

Archive:  ppyolo_tiny_650e_coco.zip
  inflating: home/aistudio/PaddleDetection/inference_model/ppyolo_tiny_650e_coco/model.pdiparams  
  inflating: home/aistudio/PaddleDetection/inference_model/ppyolo_tiny_650e_coco/infer_cfg.yml  
  inflating: home/aistudio/PaddleDetection/inference_model/ppyolo_tiny_650e_coco/model.pdiparams.info  
  inflating: home/aistudio/PaddleDetection/inference_model/ppyolo_tiny_650e_coco/model.pdmodel  


In [ ]:
%cd /home/aistudio/solutionSubmit/

/home/aistudio/solutionSubmit


In [ ]:
#使用predict.py 进行预测,只是用来模拟测试平台的运行看运行时间
!python predict.py data.txt result.json

python: can't open file 'predict.py': [Errno 2] No such file or directory


# 注意事项
 1. class_id 是否加一取决于模型是否包括背景, 包括就不用加,反之要加一,在visualize.py改
 2. 下载整个solutionSubmit文件夹,要删掉data.txt和result.json,因为该文件是生成的测试文件
 3. 重新打包要进入到包含predict.py, env, model,train.py的文件夹把四个文件一起打包, 不要用数字命名,两次提交zip名称不能相同
 4. 本方案仅仅包括检测, 而且模型是没有经过训练的模型, 所以直接提交精度很低.主要起到的是抛砖引玉的作用.希望参赛者能够发挥自己所长,找到一个又快又好的模型


# 参赛链接

https://aistudio.baidu.com/aistudio/competition/detail/91

希望大家踊跃参加,不被提交形式限制才能的发挥!
最后预祝大家获取好成绩

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 